# Extracción de datos desde APIs con Python


## Conceptos básicos de APIs REST y consumo de APIs en Python

### ¿Qué son las API?

- Son un conjuntos de reglas y protocolos que permiten a diferentes aplicaciones comunicarse entre sí. 
- Se utilizan para acceder a datos de servicios web, aplicaciones en la nube, redes sociales, entre otros. 
- Proporcionan una forma estructurada y segura de obtener datos de sistemas externos.

![API](../Images/API.png)

## Extracción de datos JSON desde APIs públicas y privadas

Consumo de la API de [mockaroo](https://www.mockaroo.com/), que es una herramienta que permite generar datos de prueba en diferentes formatos, como JSON, CSV, SQL y Excel.

In [3]:
import requests
headers = {'X-Api-Key': 'd67d54e0'}
response = requests.get('https://my.api.mockaroo.com/mkt_data_cf.json', headers=headers)

In [9]:
response.json()

[{'Fecha': '8/20/2020',
  'Nombre de la Campaña': 'Rev',
  'Region': 'Africa',
  'Clicks': 2975,
  'Impresiones': 87782,
  'Views': 692418,
  'Costo': 357},
 {'Fecha': '1/16/2020',
  'Nombre de la Campaña': 'Rev',
  'Region': 'Europe',
  'Clicks': 5769,
  'Impresiones': 31442,
  'Views': 64834,
  'Costo': 290},
 {'Fecha': '7/8/2020',
  'Nombre de la Campaña': 'Dr',
  'Region': 'North America',
  'Clicks': 6125,
  'Impresiones': 31220,
  'Views': 67623,
  'Costo': 994},
 {'Fecha': '8/10/2021',
  'Nombre de la Campaña': 'Dr',
  'Region': 'South America',
  'Clicks': 3001,
  'Impresiones': 51496,
  'Views': 155248,
  'Costo': 533},
 {'Fecha': '4/24/2021',
  'Nombre de la Campaña': 'Honorable',
  'Region': 'Europe',
  'Clicks': 6985,
  'Impresiones': 36576,
  'Views': 980070,
  'Costo': 923},
 {'Fecha': '5/7/2022',
  'Nombre de la Campaña': 'Ms',
  'Region': 'Africa',
  'Clicks': 5229,
  'Impresiones': 10852,
  'Views': 608811,
  'Costo': 600},
 {'Fecha': '12/31/2020',
  'Nombre de la Camp

### Pandas JSON

In [13]:
import pandas as pd
from io import StringIO

In [14]:
pd.read_json(StringIO(response.text))

,Fecha,Nombre de la Campaña,Region,Clicks,Impresiones,Views,Costo
0,8/20/2020,Rev,Africa,2975,87782,692418,357
1,1/16/2020,Rev,Europe,5769,31442,64834,290
2,7/8/2020,Dr,North America,6125,31220,67623,994
3,8/10/2021,Dr,South America,3001,51496,155248,533
4,4/24/2021,Honorable,Europe,6985,36576,980070,923
5,5/7/2022,Ms,Africa,5229,10852,608811,600
6,12/31/2020,Honorable,South America,9728,82941,518395,431
7,10/20/2022,Mrs,Europe,8976,40852,672668,975
8,6/4/2022,Mrs,Australia,291,60266,72736,920
9,11/21/2021,Rev,Africa,7722,92893,501754,24


In [73]:
import os
import json

path_data = os.path.join(os.getcwd(), 'data') 
ejemplo1_path = os.path.join(path_data, 'ejemplo1.json')
ejemplo2_path = os.path.join(path_data, 'ejemplo2.json')
ejemplo3_path = os.path.join(path_data, 'ejemplo3.json')

#### Ejemplo 1

In [32]:
pd.read_json(ejemplo1_path)

,nombre,productos
0,Tienda de Electrónica,"{'nombre_prod': 'Laptop', 'precio': 800, 'marc..."
1,Tienda de Electrónica,"{'nombre_prod': 'Teléfono', 'precio': 500, 'ma..."


#### json_normalize

**¿Qué hace `pd.json_normalize`?**

En esencia, esta función de Pandas toma datos JSON potencialmente complejos y anidados, y los transforma en un DataFrame plano y estructurado, que es mucho más fácil de manipular y analizar.

**Desglose de los parámetros:**

* `data`: Este es el JSON que contiene tus datos.

* `record_path='productos'`: Aquí le estás diciendo a Pandas: "Dentro de mi diccionario `data`, busca una lista llamada 'productos'. Cada elemento dentro de esa lista representa una fila en mi DataFrame final".

* `meta=['nombre']`:  Con esto indicas: "Además de los datos dentro de 'productos', quiero incluir una columna adicional llamada 'nombre' en mi DataFrame. El valor de esta columna será el valor asociado a la clave 'nombre' en el nivel superior de mi diccionario `data`".  En nuestro ejemplo, esto significa que cada fila tendrá el nombre de la tienda ("Tienda de Electrónica") junto con los detalles del producto.


In [35]:
data = json.load(open(ejemplo1_path))

pd.json_normalize(data, record_path='productos', meta=['nombre'])

,nombre_prod,precio,marca,nombre
0,Laptop,800,Marca A,Tienda de Electrónica
1,Teléfono,500,Marca B,Tienda de Electrónica


#### Ejemplo 2

In [42]:
pd.read_json(ejemplo2_path)

,usuario
nombre,Juan
edad,30
direcciones,"[{'calle': 'Calle Principal 123', 'ciudad': 'C..."
compras,"[{'producto': 'Libro', 'precio': 20}, {'produc..."


In [74]:
data2 = json.load(open(ejemplo2_path))
pd.json_normalize(
    data2
    )

,usuario.nombre,usuario.edad,usuario.direcciones,usuario.compras
0,Juan,30,"[{'calle': 'Calle Principal 123', 'ciudad': 'C...","[{'producto': 'Libro', 'precio': 20, 'fecha': ..."


In [75]:
df_direcciones = pd.json_normalize(
    data2['usuario'],
    record_path=['direcciones'],
    meta=['nombre', 'edad',]
    )
df_direcciones

,calle,ciudad,id,nombre,edad
0,Calle Principal 123,Ciudad A,1,Juan,30
1,Avenida Secundaria 456,Ciudad B,2,Juan,30


In [76]:
df_compras = pd.json_normalize(
    data2['usuario'],
    record_path=['compras'],
    meta=['nombre', 'edad',]
    )
df_compras

,producto,precio,fecha,id,nombre,edad
0,Libro,20,2020-01-01,1,Juan,30
1,Película,15,2020-01-02,2,Juan,30


#### Ejemplo 3

In [77]:
pd.read_json(ejemplo3_path)

,escuela
nombre,Escuela Primaria
grados,"[{'nombre': 'Primer Grado', 'alumnos': [{'nomb..."


In [87]:
data3 = json.load(open(ejemplo3_path))
pd.json_normalize(
    data3['escuela'],)

,nombre,grados
0,Escuela Primaria,"[{'nombre': 'Primer Grado', 'alumnos': [{'nomb..."


In [105]:
pd.json_normalize(
    data3['escuela']['grados'],
        record_path='alumnos',
        meta=['nombre'],
        meta_prefix='grado_'
)




,nombre,edad,grado_nombre
0,Ana,6,Primer Grado
1,Pedro,7,Primer Grado
2,María,7,Segundo Grado
3,Luis,8,Segundo Grado
